# Topic Modeling with BERTopic on Parliamentary Speeches - Google Colab Version

This notebook is optimized for Google Colab with GPU acceleration. It implements a complete pipeline from data loading to topic modeling:

1. **Data Loading** - Loads the AT_original_complete.pkl file and processes it
2. **Data Filtering** - Creates processed version for topic modeling
3. **Dual Embedding** - Speech-level and segment-level embeddings
4. **Semantic Segmentation** - Similarity-based boundary detection  

## Key Approach - Dual Embedding Strategy:
- **First embedding**: Individual speeches using raw text (for segmentation)
- **Second embedding**: Concatenated segment texts (for topic modeling)
- **Why twice?** Re-embedding captures full discourse coherence vs. averaging individual embeddings
- **Raw text used throughout** for better semantic capture

In [ ]:
# === GOOGLE COLAB SETUP ===
# Mount Google Drive to access your data
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install sentence-transformers bertopic umap-learn hdbscan tqdm openai python-dotenv

# Check GPU availability and optimize for A100
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # Optimize for A100
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    print("No GPU detected - will use CPU (slower)")

# Import all required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import gc

# Set random seed for reproducibility
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("Setup complete! ✓")

ModuleNotFoundError: No module named 'google.colab'

## Data Loading and Processing

Load the original complete data from Google Drive and create the processed version for topic modeling.

In [ ]:
# === DATA LOADING AND PROCESSING FOR GOOGLE COLAB ===

# Path to your file in Google Drive (update if needed)
data_folder = '/content/drive/MyDrive/data folder/data/'
data_path = f'{data_folder}AT_original_complete.pkl'

AT_original_df = pd.read_pickle(data_path)
print(f"✅ Loaded original complete data: {AT_original_df.shape}")
print(f"Columns: {list(AT_original_df.columns)}")

# Filter out short speeches for segmentation and embedding
long_df = AT_original_df[~AT_original_df['Is_Too_Short']].copy()
short_df = AT_original_df[AT_original_df['Is_Too_Short']].copy()
print(f"Long speeches for segmentation: {len(long_df):,}")
print(f"Short speeches to assign after segmentation: {len(short_df):,}")

✅ Loaded original complete data: (231752, 27)
Columns: ['Sitting_ID', 'Speech_ID', 'Title', 'Date', 'Body', 'Term', 'Session', 'Meeting', 'Sitting', 'Agenda', 'Subcorpus', 'Lang', 'Speaker_role', 'Speaker_MP', 'Speaker_minister', 'Speaker_party', 'Speaker_party_name', 'Party_status', 'Party_orientation', 'Speaker_ID', 'Speaker_name', 'Speaker_gender', 'Speaker_birth', 'Text', 'Word_Count', 'Is_Too_Short', 'Is_Filtered']

📈 Ready for topic modeling: 231,752 speeches

📈 Ready for topic modeling: 231,752 speeches


## Embedding and Segmentation Functions (GPU Optimized)

These functions are optimized for GPU acceleration and handle the dual-embedding approach:
1. **Speech-level embeddings** for similarity-based segmentation  
2. **Segment-level embeddings** for final topic modeling

In [ ]:
# === EMBEDDING FUNCTIONS (A100 OPTIMIZED) ===
from sklearn.metrics.pairwise import cosine_similarity
from scipy.signal import find_peaks
from sentence_transformers import SentenceTransformer
import torch
import time
import gc
from tqdm import tqdm
import pickle
import os

def load_embedding_model(model_name="BAAI/bge-m3", device=None):
    """Load embedding model optimized for A100 GPU."""
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

    print(f"Loading embedding model: {model_name} on {device}")
    start_time = time.time()

    try:
        model = SentenceTransformer(
            model_name, 
            device=device, 
            trust_remote_code=True,
            model_kwargs={'torch_dtype': torch.float16}  # Use FP16 for A100
        )
        # Optimize for A100
        if device == 'cuda':
            model.half()  # Use FP16 for faster inference on A100
        print(f"✓ Model loaded in {time.time() - start_time:.2f} seconds")
        return model
    except Exception as e:
        print(f"❌ Error loading {model_name}: {e}")
        raise e

def chunk_text_tokenwise(text, tokenizer, chunk_size=4096, overlap=1024):
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    starts = list(range(0, len(token_ids), chunk_size - overlap))
    for start in starts:
        end = min(start + chunk_size, len(token_ids))
        chunk_ids = token_ids[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append((chunk_text, len(chunk_ids)))
    return chunks

def weighted_mean(embeddings, weights):
    embeddings = np.stack(embeddings)
    weights = np.array(weights)
    weights = weights / weights.sum()
    return np.average(embeddings, axis=0, weights=weights)

def embed_text_bge(text, model, tokenizer):
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    if len(token_ids) <= 8192:
        return model.encode(text, convert_to_tensor=False, show_progress_bar=False)
    else:
        chunks = chunk_text_tokenwise(text, tokenizer, chunk_size=4096, overlap=1024)
        chunk_texts, chunk_lengths = zip(*chunks)
        # Use larger batch size for A100 and suppress progress bar
        chunk_embeddings = model.encode(
            list(chunk_texts), 
            batch_size=128, 
            convert_to_tensor=False,
            show_progress_bar=False  # Suppress internal progress bar
        )
        return weighted_mean(chunk_embeddings, chunk_lengths)

def save_checkpoint(data, checkpoint_path):
    """Save checkpoint for resuming."""
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"💾 Checkpoint saved: {checkpoint_path}")

def load_checkpoint(checkpoint_path):
    """Load checkpoint for resuming."""
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            data = pickle.load(f)
        print(f"📂 Checkpoint loaded: {checkpoint_path}")
        return data
    return None

def generate_speech_embeddings_for_segmentation(
    df, text_column='Text', model_name="BAAI/bge-m3", 
    batch_size=64, checkpoint_freq=5000
):
    """
    Generate BGE-m3 embeddings with A100 optimization and checkpointing.
    """
    print("=" * 60)
    print("SPEECH EMBEDDINGS: BGE-m3 optimized for A100")
    print("=" * 60)
    print(f"Processing {len(df)} speeches with batch_size={batch_size}")

    # Setup checkpointing
    checkpoint_dir = '/content/drive/MyDrive/checkpoints/'
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = f'{checkpoint_dir}speech_embeddings_checkpoint.pkl'
    
    # Try to load existing checkpoint
    checkpoint_data = load_checkpoint(checkpoint_path)
    if checkpoint_data:
        start_idx = checkpoint_data['last_processed_idx'] + 1
        embeddings = checkpoint_data['embeddings']
        print(f"🔄 Resuming from index {start_idx}")
    else:
        start_idx = 0
        embeddings = []

    model = SentenceTransformer(model_name, device="cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        model.half()  # FP16 for A100
    tokenizer = model.tokenizer
    model.max_seq_length = 8192

    texts = df[text_column].astype(str).values
    
    # Process in batches with larger batch size for A100
    total_batches = (len(texts) - start_idx + batch_size - 1) // batch_size
    
    with tqdm(total=len(texts), initial=start_idx, desc="🚀 Embedding speeches", unit="speech") as pbar:
        for i in range(start_idx, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Process batch - handle long texts individually but in batches where possible
            batch_embeddings = []
            short_texts = []
            short_indices = []
            
            for j, text in enumerate(batch_texts):
                token_count = len(tokenizer.encode(text, add_special_tokens=False))
                if token_count <= 8192:
                    short_texts.append(text)
                    short_indices.append(j)
                else:
                    # Handle long text individually - suppress any internal progress
                    emb = embed_text_bge(text, model, tokenizer)
                    batch_embeddings.append((j, emb))
            
            # Batch process short texts with larger batch size
            if short_texts:
                short_embeddings = model.encode(
                    short_texts, 
                    batch_size=min(128, len(short_texts)),  # A100 optimized batch size
                    convert_to_tensor=False,
                    show_progress_bar=False  # Suppress internal progress bar
                )
                for idx, emb in zip(short_indices, short_embeddings):
                    batch_embeddings.append((idx, emb))
            
            # Sort by original order and add to results
            batch_embeddings.sort(key=lambda x: x[0])
            embeddings.extend([emb for _, emb in batch_embeddings])
            
            # Update progress (only once per batch)
            pbar.update(len(batch_texts))
            
            # Save checkpoint periodically (suppress checkpoint messages)
            if (i + batch_size) % checkpoint_freq == 0:
                checkpoint_data = {
                    'embeddings': embeddings,
                    'last_processed_idx': i + len(batch_texts) - 1
                }
                with open(checkpoint_path, 'wb') as f:
                    pickle.dump(checkpoint_data, f)
                # Only show checkpoint message every 20k speeches
                if (i + batch_size) % (checkpoint_freq * 4) == 0:
                    print(f"\n💾 Checkpoint: {i + batch_size:,}/{len(texts):,} speeches processed")
                
            # Clear GPU cache periodically
            if (i + batch_size) % (checkpoint_freq * 2) == 0:
                torch.cuda.empty_cache()
                gc.collect()

    # Clean up checkpoint
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        print("🧹 Checkpoint cleaned up")

    df_with_embeddings = df.copy()
    df_with_embeddings['Speech_Embeddings'] = embeddings
    return df_with_embeddings

# === IMPROVED PARLIAMENTARY SEGMENTATION FUNCTIONS ===
import warnings
warnings.filterwarnings('ignore')

def parliamentary_segment_speeches(df, window_size=3, min_segment_size=3):
    """
    Parliamentary segmentation with multi-scale analysis and chairperson agenda detection
    """
    segment_ids = []
    segmentation_metrics = []
    
    # Get unique sittings for progress tracking
    unique_sittings = df['Sitting_ID'].unique()
    print(f"🔄 Processing {len(unique_sittings)} sittings...")
    
    for sitting_id in tqdm(unique_sittings, desc="Segmenting sittings", unit="sitting"):
        group = df[df['Sitting_ID'] == sitting_id]
        sitting_length = len(group)
        
        if sitting_length < min_segment_size:
            # Very small sitting - one segment
            sitting_segments = [f"{sitting_id}_seg_0"] * len(group)
            segment_ids.extend(sitting_segments)
            segmentation_metrics.append({
                'sitting_id': sitting_id,
                'sitting_length': sitting_length,
                'num_segments': 1,
                'avg_segment_size': sitting_length,
                'boundaries_found': 0,
                'agenda_boundaries': 0
            })
            continue
        
        embeddings = np.array(group['Speech_Embeddings'].tolist())
        
        # --- NEW: Flexible target_segments formula ---
        target_segments = max(2, int(np.ceil(sitting_length / 25)))
        threshold_percentile = 40
        
        # === CHAIRPERSON AGENDA DETECTION ===
        agenda_boundaries = set()
        agenda_signals = []
        
        for i, (idx, row) in enumerate(group.iterrows()):
            agenda_score = 0
            
            # Strong signal for chairperson with agenda mentions
            if row['Speaker_role'] == 'Chairperson':
                text = str(row['Text']).lower()
                
                if 'agenda item' in text:
                    agenda_score = 1.0  # Strongest signal
                elif 'agenda' in text:
                    agenda_score = 0.7  # Strong signal
                elif i == 0:  # First speech by chairperson (session start)
                    agenda_score = 0.3  # Mild signal
            
            agenda_signals.append(agenda_score)
            
            # Add strong agenda boundaries
            if agenda_score >= 0.7 and i >= min_segment_size and (sitting_length - i) >= min_segment_size:
                agenda_boundaries.add(i)
        
        # === MULTI-SCALE SIMILARITY ANALYSIS ===
        similarity_signals = {}
        
        # 1. Primary windowed similarity
        similarities = []
        for i in range(len(embeddings) - window_size):
            window1 = np.mean(embeddings[i:i + window_size], axis=0)
            window2 = np.mean(embeddings[i + window_size:i + 2*window_size], axis=0)
            
            sim = cosine_similarity(
                window1.reshape(1, -1),
                window2.reshape(1, -1)
            )[0][0]
            similarities.append(sim)
        
        similarity_signals['primary'] = np.array(similarities)
        
        # 2. Point-to-point similarity for fine-grained detection
        if len(embeddings) > 6:
            point_sims = []
            for i in range(len(embeddings) - 1):
                sim = cosine_similarity(
                    embeddings[i].reshape(1, -1),
                    embeddings[i + 1].reshape(1, -1)
                )[0][0]
                point_sims.append(sim)
            
            # Align with primary signal
            point_sims = np.array(point_sims)
            if len(point_sims) > len(similarities):
                point_sims = point_sims[:len(similarities)]
            elif len(point_sims) < len(similarities):
                padding = len(similarities) - len(point_sims)
                point_sims = np.pad(point_sims, (0, padding), mode='edge')
            
            similarity_signals['point'] = point_sims
        
        # 3. Gradient-based change detection
        if len(embeddings) > 10:
            trajectory = []
            for i in range(1, len(embeddings)):
                displacement = np.linalg.norm(embeddings[i] - embeddings[i-1])
                trajectory.append(float(displacement))
            
            trajectory = np.array(trajectory, dtype=np.float64)
            if len(trajectory) > 3:
                try:
                    from scipy.ndimage import uniform_filter1d
                    smoothed = uniform_filter1d(trajectory.astype(np.float64), size=3)
                    gradient = np.gradient(smoothed)
                    
                    # Align with similarities
                    if len(gradient) > len(similarities):
                        gradient = gradient[:len(similarities)]
                    elif len(gradient) < len(similarities):
                        padding = len(similarities) - len(gradient)
                        gradient = np.pad(gradient, (0, padding), mode='edge')
                    
                    similarity_signals['gradient'] = gradient
                except:
                    pass
        
        if len(similarity_signals['primary']) == 0:
            sitting_segments = [f"{sitting_id}_seg_0"] * len(group)
            segment_ids.extend(sitting_segments)
            segmentation_metrics.append({
                'sitting_id': sitting_id,
                'sitting_length': sitting_length,
                'num_segments': 1,
                'avg_segment_size': sitting_length,
                'boundaries_found': 0,
                'agenda_boundaries': 0
            })
            continue
        
        # === BOUNDARY DETECTION ===
        candidate_boundaries = set()
        
        # 1. Add agenda boundaries (highest priority)
        candidate_boundaries.update(agenda_boundaries)
        
        # 2. Find boundaries from primary similarity drops
        primary_sims = similarity_signals['primary']
        threshold = np.percentile(primary_sims, threshold_percentile)
        
        for i in range(len(primary_sims)):
            if (primary_sims[i] < threshold and 
                i >= min_segment_size and 
                (len(group) - i - window_size) >= min_segment_size):
                candidate_boundaries.add(i + window_size)
        
        # 3. Add from point-to-point analysis
        if 'point' in similarity_signals:
            point_threshold = np.percentile(similarity_signals['point'], threshold_percentile - 10)
            for i in range(len(similarity_signals['point'])):
                if (similarity_signals['point'][i] < point_threshold and 
                    i >= min_segment_size and 
                    (len(group) - i) >= min_segment_size):
                    candidate_boundaries.add(i)
        
        # 4. Add from gradient analysis
        if 'gradient' in similarity_signals:
            gradient = similarity_signals['gradient']
            gradient_threshold = np.percentile(np.abs(gradient), 75)
            for i in range(len(gradient)):
                if (np.abs(gradient[i]) > gradient_threshold and 
                    i >= min_segment_size and 
                    (len(group) - i) >= min_segment_size):
                    candidate_boundaries.add(i)
        
        candidates = sorted(list(candidate_boundaries))
        
        # === BOUNDARY SELECTION WITH AGENDA PRIORITIZATION ===
        boundaries = []
        if candidates:
            if len(candidates) <= target_segments - 1:
                boundaries = candidates
            else:
                # Score candidates with agenda boost
                candidate_scores = []
                for c in candidates:
                    score = 0
                    
                    # Agenda boost (highest priority)
                    if c < len(agenda_signals):
                        score += agenda_signals[c] * 5.0  # Very high weight for agenda
                    
                    # Primary similarity score
                    if c - window_size >= 0 and c - window_size < len(primary_sims):
                        score += (1 - primary_sims[c - window_size]) * 2.0
                    
                    # Point similarity score
                    if 'point' in similarity_signals and c < len(similarity_signals['point']):
                        score += (1 - similarity_signals['point'][c]) * 1.5
                    
                    # Gradient score
                    if 'gradient' in similarity_signals and c < len(similarity_signals['gradient']):
                        score += np.abs(similarity_signals['gradient'][c]) * 1.0
                    
                    candidate_scores.append((c, score))
                
                # Select top scoring boundaries
                candidate_scores.sort(key=lambda x: x[1], reverse=True)
                boundaries = sorted([c for c, _ in candidate_scores[:target_segments-1]])
        
        # === BOUNDARY VALIDATION ===
        validated_boundaries = []
        for boundary in boundaries:
            if not validated_boundaries or (boundary - validated_boundaries[-1]) >= min_segment_size:
                validated_boundaries.append(boundary)
        
        boundaries = validated_boundaries
        
        # Assign segment IDs
        current_segment = 0
        sitting_segments = []
        
        for i in range(len(group)):
            if i > 0 and (i - 1) in boundaries:
                current_segment += 1
            sitting_segments.append(f"{sitting_id}_seg_{current_segment}")
        
        segment_ids.extend(sitting_segments)
        
        # Store metrics
        num_segments = len(set(sitting_segments))
        agenda_bound_count = len([b for b in boundaries if b in agenda_boundaries])
        
        segmentation_metrics.append({
            'sitting_id': sitting_id,
            'sitting_length': sitting_length,
            'num_segments': num_segments,
            'avg_segment_size': sitting_length / num_segments,
            'boundaries_found': len(boundaries),
            'agenda_boundaries': agenda_bound_count,
            'target_segments': target_segments,
            'candidate_boundaries': len(candidates),
            'signals_used': len(similarity_signals) + 1  # +1 for agenda signals
        })
    
    df['Segment_ID'] = segment_ids
    return df, segmentation_metrics

print("✓ Enhanced parliamentary segmentation with agenda detection loaded")

✓ Embedding and segmentation functions loaded


In [ ]:
# === RUN THE ENHANCED PIPELINE ===

print("🚀 Starting A100-optimized embedding pipeline with enhanced segmentation...")
print(f"💻 Using: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"📊 Processing {len(long_df)} speeches for segmentation")

# Clear any existing cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

try:
    # STEP 1: Generate speech-level embeddings with A100 optimization
    print("\n🔄 Generating speech-level embeddings...")
    df_with_speech_embeddings = generate_speech_embeddings_for_segmentation(
        long_df, 
        text_column='Text',
        batch_size=64,  # A100 optimized batch size
        checkpoint_freq=10000  # Checkpoint every 10k speeches
    )
    print("✅ Speech-level embeddings generated!")
    
    # Clear cache before segmentation
    torch.cuda.empty_cache()
    gc.collect()

    # STEP 2: Enhanced parliamentary segmentation with agenda detection
    print("\n🏛️ Running enhanced parliamentary segmentation...")
    df_segmented, seg_metrics = parliamentary_segment_speeches(
        df_with_speech_embeddings, 
        window_size=5,        
        min_segment_size=3    # Smaller minimum for more segments
    )
    
    # Display segmentation results
    metrics_df = pd.DataFrame(seg_metrics)
    print(f"\n✅ Enhanced segmentation complete!")
    print(f"📊 Results:")
    print(f"  • Total speeches processed: {len(df_segmented):,}")
    print(f"  • Unique segments created: {df_segmented['Segment_ID'].nunique():,}")
    print(f"  • Average speeches per segment: {len(df_segmented) / df_segmented['Segment_ID'].nunique():.1f}")
    print(f"  • Average segments per sitting: {metrics_df['num_segments'].mean():.1f}")
    print(f"  • Agenda boundaries used: {metrics_df['agenda_boundaries'].sum()}")

    # STEP 3: Assign short speeches to nearest segment (FIXED)
    print("\n🔄 Assigning short speeches to segments...")
    def assign_short_speeches(short_df, segmented_df):
        """Assign short speeches to segments based on their original order within sittings."""
        assigned = []
        for sitting_id, group in short_df.groupby('Sitting_ID'):
            seg_group = segmented_df[segmented_df['Sitting_ID'] == sitting_id]
            if seg_group.empty:
                # If no segments in this sitting, create a default segment
                default_segment = f"{sitting_id}_seg_0"
                assigned.extend([default_segment] * len(group))
                continue
            
            # Get unique segments for this sitting in order
            segments_in_sitting = seg_group['Segment_ID'].unique()
            
            # For each short speech, assign to the first available segment
            # This is a simple strategy - you could make it more sophisticated
            for idx, row in group.iterrows():
                # Assign to first segment (could be improved with better logic)
                assigned.append(segments_in_sitting[0])
        
        short_df = short_df.copy()
        short_df['Segment_ID'] = assigned
        return short_df

    short_df_assigned = assign_short_speeches(short_df, df_segmented)
    df_all = pd.concat([df_segmented, short_df_assigned], ignore_index=True)

    # STEP 4: Generate segment-level embeddings with A100 optimization
    print("\n🔄 Generating segment-level embeddings...")
    df_final = generate_segment_embeddings(
        df_all, 
        text_column='Text', 
        segment_id_column='Segment_ID',
        batch_size=32  # A100 optimized for segments
    )
    print("✅ Segment-level embeddings mapped!")

    # STEP 5: Save final output
    output_path = f"{data_folder}AT_with_embeddings_final.pkl"
    df_final.to_pickle(output_path)
    print(f"\n💾 Saved final dataframe: {output_path}")
    print(f"📊 Final shape: {df_final.shape}")
    print(f"🎯 Segments created: {df_final['Segment_ID'].nunique()}")
    
    # Final cleanup
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 Memory cleaned up")

except Exception as e:
    print(f"❌ Error in pipeline: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# === SAVE RESULTS (OPTIONAL) ===
# Uncomment to save the final results

print("💾 Saving final results...")
final_embeddings.to_pickle('data folder/data/AT_with_22_topics_final.pkl')
final_topic_info.to_pickle('data folder/data/topic_info_22_categories.pkl')
category_stats.to_pickle('data folder/data/category_statistics.pkl')
print("✅ Results saved!")

print("\n🎉 Analysis complete! Ready to run.")